In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 什么是交叉验证
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

In [4]:
iris = load_iris()
data = iris.data
target = iris.target

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

1.0

In [8]:
# 因为数据切割产生的影响, 造成模型评估不能客观的反应模型的性能
# 如何才能消除因为数据切割对算法性能产生的影响

In [9]:
from sklearn.model_selection import KFold, StratifiedKFold

In [10]:
# KFold 普通的交叉验证, 通过把数据分成k折
kfold = KFold(n_splits=5, shuffle=True)

In [29]:
# 返回训练数据和测试数据的索引
scores = []
for train, test in kfold.split(data, target):
#     print(train)
#     print(test)
#     print('----------------------------')
    # 取数据
    X_train = data[train]
    y_train = target[train]
    X_test = data[test]
    y_test = target[test]
    
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(score)
    scores.append(score)
print(np.array(scores).mean())

0.9666666666666667
0.9666666666666667
0.9666666666666667
0.9666666666666667
0.9333333333333333
0.96


In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [12]:
# 返回训练数据和测试数据的索引
scores = []
for train, test in skf.split(data, target):
#     print(train)
#     print(test)
#     print('----------------------------')
    # 取数据
    X_train = data[train]
    y_train = target[train]
    X_test = data[test]
    y_test = target[test]
    
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    scores.append(score)
    print(score)
print(np.array(scores).mean())

0.9333333333333333
0.9666666666666667
0.9666666666666667
1.0
0.9666666666666667
0.9666666666666666


In [34]:
# 调整k值和p以求达到一个更好的效果
# 需要计算每一次参数算法表现
for k in range(2, 20):
    for p in range(1, 3):
        scores = []
        for train, test in kfold.split(data, target):
        #     print(train)
        #     print(test)
        #     print('----------------------------')
            # 取数据
            X_train = data[train]
            y_train = target[train]
            X_test = data[test]
            y_test = target[test]

            knn = KNeighborsClassifier(k, p=p)
            knn.fit(X_train, y_train)
            score = knn.score(X_test, y_test)
            scores.append(score)
#             print(score)
        print(k, p, np.array(scores).mean())

2 1 0.9266666666666667
2 2 0.9333333333333333
3 1 0.9533333333333334
3 2 0.9733333333333334
4 1 0.9400000000000001
4 2 0.96
5 1 0.9666666666666668
5 2 0.9533333333333334
6 1 0.9533333333333334
6 2 0.9666666666666666
7 1 0.9466666666666667
7 2 0.9800000000000001
8 1 0.9600000000000002
8 2 0.9666666666666666
9 1 0.96
9 2 0.9666666666666666
10 1 0.9533333333333334
10 2 0.96
11 1 0.9466666666666667
11 2 0.9800000000000001
12 1 0.9533333333333334
12 2 0.9733333333333334
13 1 0.9400000000000001
13 2 0.9733333333333334
14 1 0.96
14 2 0.9666666666666668
15 1 0.9533333333333334
15 2 0.9733333333333334
16 1 0.9533333333333334
16 2 0.9466666666666667
17 1 0.9533333333333334
17 2 0.9666666666666668
18 1 0.9533333333333334
18 2 0.9666666666666666
19 1 0.96
19 2 0.96


In [35]:
from sklearn.model_selection import GridSearchCV  # 网格搜索 CV : cross validation

In [37]:
knn = KNeighborsClassifier()
# 字典的key必须和算法中的参数一模一样
param_grid = {
    'n_neighbors': range(2, 20),
    'p': [1, 2]
}
gv = GridSearchCV(estimator=knn, param_grid=param_grid, n_jobs=-1, cv=5, verbose=2)

In [38]:
gv.fit(data, target)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(2, 20), 'p': [1, 2]}, verbose=2)

In [39]:
# 最佳参数
gv.best_params_

{'n_neighbors': 6, 'p': 2}

In [40]:
# 最佳得分
gv.best_score_

0.9800000000000001

In [43]:
# 使用最佳参数训练出来的模型
gv.best_estimator_

KNeighborsClassifier(n_neighbors=6)

In [44]:
# 或者之家使用gv, gv就相当于是最佳参数训练出来的模型的代理对象
gv.score(data, target)

0.9733333333333334